In [14]:
import pandas as pd

In [15]:
path_directory = "/home/oem/PycharmProjects/RealEstate_Data_Pipeline/processed_data/.ipynb_checkpoints/combined_data-checkpoint.csv"
path_direct2 = "/home/oem/PycharmProjects/RealEstate_Data_Pipeline/spark_query/dataframe1.csv"

In [16]:
#Loadin function
def load_data(path_directory, path_direct2):
    df1 = pd.read_csv(path_directory, low_memory=False)
    df2 = pd.read_csv(path_direct2, low_memory=False)
    
    for cols in df1.columns:
        if df1[cols].apply(type).nunique() > 1:
            print(f"These col: {cols} has mixed data type in df1")
    
    for cols in df2.columns:
        if df2[cols].apply(type).nunique() > 1:
            print(f"These col: {cols} has mixed data type in df2")
    return df1, df2

In [17]:
df1, df2 = load_data(path_directory, path_direct2)

These col: StateName has mixed data type in df1
These col: State has mixed data type in df1
These col: Metro has mixed data type in df1
These col: CountyName has mixed data type in df1
These col: City has mixed data type in df1
These col: City has mixed data type in df2
These col: Metro has mixed data type in df2


In [18]:
df1.dtypes

RegionID               int64
SizeRank               int64
RegionName            object
RegionType            object
StateName             object
                      ...   
2024-09-30           float64
2024-10-31           float64
StateCodeFIPS        float64
MunicipalCodeFIPS    float64
City                  object
Length: 309, dtype: object

In [19]:
def correcting_dtypes(df1, df2):
    cols_affected = [
        "State",
        "City",
        "Metro",
        "CountyName",
        "StateName"
    ]

    for col in cols_affected:
        if col in df1.columns:
            df1[col] = df1[col].astype(str)
        if col in df2.columns:
            df2[col] = df2[col].astype(str)

    return df1, df2

In [20]:
df1, df2 = correcting_dtypes(df1, df2)
df1.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,2000-01-31,2000-02-29,...,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,StateCodeFIPS,MunicipalCodeFIPS,City
0,6181,0,New York,city,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,353158.446420,354933.117086,...,1.203104e+06,1.214107e+06,1.218648e+06,1.220656e+06,1.221700e+06,1.224398e+06,1.224366e+06,NaN,NaN,nan
1,12447,1,Los Angeles,city,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,443984.627814,444399.053109,...,1.920614e+06,1.921324e+06,1.916270e+06,1.915412e+06,1.922208e+06,1.937383e+06,1.949880e+06,NaN,NaN,nan
2,39051,2,Houston,city,TX,TX,"Houston-The Woodlands-Sugar Land, TX",Harris County,242987.400664,242873.804631,...,5.173619e+05,5.195665e+05,5.204305e+05,5.202132e+05,5.202257e+05,5.210260e+05,5.220773e+05,NaN,NaN,nan
3,17426,3,Chicago,city,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,181915.347907,181990.844464,...,4.277582e+05,4.304698e+05,4.322804e+05,4.335986e+05,4.353049e+05,4.372507e+05,4.386567e+05,NaN,NaN,nan
4,6915,4,San Antonio,city,TX,TX,"San Antonio-New Braunfels, TX",Bexar County,204044.849078,204382.271133,...,4.513202e+05,4.520653e+05,4.521270e+05,4.515137e+05,4.510616e+05,4.509712e+05,4.511448e+05,NaN,NaN,nan


In [21]:
# Droping abnormal columns
def clean_data(df):
    df = df.drop(columns=['City', 'StateCodeFIPS', "MunicipalCodeFIPS"], axis=1)
    print(df.duplicated(keep='first').sum())
    df.fillna(0, inplace=True)
    
    return df

In [22]:
df1 = clean_data(df1)

0


In [23]:
def calculate_quarter_prices(df, quarter, year_start=2000, year_end=2024):

    quarter_months = {
        "first": ["01-31", "02-28", "03-31"],
        "second": ["04-30", "05-31", "06-30"],
        "third": ["07-31", "08-31", "09-30"],
        "fourth": ["10-31", "11-30", "12-31"]
    }

    if quarter not in quarter_months:
        raise ValueError(f"Invalid quarter name: {quarter}. Must be one of: {list(quarter_months.keys())}")

    months = quarter_months[quarter]

    for year in range(year_start, year_end + 1):
        # Adjust February for leap years if it's the first quarter
        if quarter == "first" and pd.Timestamp(f"{year}-02-01").is_leap_year:
            months[1] = "02-29"  # Replace 02-28 with 02-29 for leap years
        else:
            months[1] = "02-28"  # Reset to 02-28 for non-leap years

        # Generate the column names for the quarter
        date_columns = [f"{year}-{month}" for month in months]

        # Check for missing columns
        missing_cols = [col for col in date_columns if col not in df.columns]
        if missing_cols:
            print(f"Warning: Missing columns for {year} {quarter}: {missing_cols}")
            continue

        # Calculate the average and add the new column
        column_name = f"{year}_{quarter}_qtr_prices"
        df[column_name] = df[date_columns].mean(axis=1).round(2)

    return df

In [24]:
def processed_quarters(df1):
    first_qtrs_df = calculate_quarter_prices(df1, quarter="first")
    second_qtrs_df= calculate_quarter_prices(df1, quarter="second")
    third_qtrs_df = calculate_quarter_prices(df1, quarter="third")
    fourth_qtrs_df = calculate_quarter_prices(df1, quarter="fourth")
    
    comm_col = df1.iloc[:, 0:8]
    drop_cols = df1.iloc[:, 0:306].columns
    
    # # Drop specified columns in each DataFrame if they exist
    first_qtrs_df.drop(columns=drop_cols, inplace=True, errors="ignore")
    second_qtrs_df.drop(columns=drop_cols, inplace=True, errors="ignore")
    third_qtrs_df.drop(columns=drop_cols, inplace=True, errors="ignore")
    fourth_qtrs_df.drop(columns=drop_cols, inplace=True, errors="ignore")

    df = pd.concat([comm_col, first_qtrs_df, second_qtrs_df, third_qtrs_df, fourth_qtrs_df], axis=1)
    
    return df
final_df = processed_quarters(df1)
final_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,2000_first_qtr_prices,2001_first_qtr_prices,...,2011_fourth_qtr_prices,2013_fourth_qtr_prices,2014_fourth_qtr_prices,2015_fourth_qtr_prices,2017_fourth_qtr_prices,2018_fourth_qtr_prices,2019_fourth_qtr_prices,2021_fourth_qtr_prices,2022_fourth_qtr_prices,2023_fourth_qtr_prices
0,6181,0,New York,city,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,354996.80,401288.24,...,608672.96,658140.10,718798.15,799754.84,952278.57,1021358.07,1019899.59,1127560.29,1194027.61,1184220.63
1,12447,1,Los Angeles,city,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,444897.42,483896.39,...,745906.50,869430.46,958830.28,973380.00,1088885.27,1299874.89,1365394.99,1670770.32,1881315.91,1930667.60
2,39051,2,Houston,city,TX,TX,"Houston-The Woodlands-Sugar Land, TX",Harris County,242686.22,244091.99,...,276178.26,303356.26,333930.88,361584.99,360952.35,361474.50,367439.86,427460.84,484078.23,503549.88
3,17426,3,Chicago,city,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,182120.47,204231.90,...,234987.75,239053.48,265726.36,285783.64,322891.28,339692.14,344274.62,392367.89,410789.75,412526.05
4,6915,4,San Antonio,city,TX,TX,"San Antonio-New Braunfels, TX",Bexar County,204382.64,201026.87,...,241684.84,257150.54,269625.94,284071.30,302202.22,311283.89,319499.51,386789.40,445405.92,455072.03


In [25]:
def create_state_dfs(df):
    # List of states
    states = [
        "TX", "CA", "NY", "FL", "IL",
        "OH", "GA", "MA", "VA", "WA",
        "PA", "NC", "CO", "MN", "IN",
        "MI", "IA", "MD", "KS", "UT", "OR"
    ]
    
    # Create a dictionary of DataFrames for each state
    state_dfs = {state: df[df["State"] == state] for state in states}
    
    return state_dfs


In [26]:
state_dfs = create_state_dfs(first_qtrs_df)

# Access specific state DataFrames
tx_df = state_dfs["TX"]
fl_df = state_dfs["FL"]
oh_df = state_dfs["OH"]

NameError: name 'first_qtrs_df' is not defined

In [ ]:
tx_df.head()

In [ ]:
#Droping columns 
col_drop = tx_df.iloc[:, 8:306]
for col in col_drop:
    if col in tx_df.columns:
        print(f"Dropped column: {col}")
        tx_df = tx_df.drop(col, axis=1)

In [ ]:
tx_df.head()